In [2]:
import pandas as pd
import os

import requests
import re
import time
import numpy as np
import subprocess
import random

In [46]:
%cd ./../outguard/instrumentation/

[Errno 2] No such file or directory: './../outguard/instrumentation/'
/home/gioacchino/Desktop/iogames/outguard/instrumentation


### poki.com game crawler

In [ ]:
LOG = "/home/gioacchino/Desktop/iogames/static_analysis/poki_games2.txt"
INFO =  "/home/gioacchino/Desktop/iogames/static_analysis/poki_game_info2.txt"

with open(LOG,'a') as fout:
    fout.write("Game_name" + '\t' + "Game_url" + '\n')


CHROME = ['google-chrome',
          '--remote-debugging-port=9222', 
          '--incognito', 
          '--allow-running-insecure-content', 
          '--headless'] 



def run_crawler(url):
    #remove all dom files
    os.system("rm /home/gioacchino/Desktop/iogames/static_analysis/PATH_DOM_GAMEPLATFORMS/*")
    
    if "://" in url:
        url = url.split("://")[1]
    if url[-1] == '/':
        url = url[:-1]
    
    print("Processing website: ", url)
    
    #run chrome
    p = subprocess.Popen(CHROME)
    
    #wait a bit
    time.sleep(5)
    
    #run node-js script for full profiling (homepage only)
    cmd = ['node', 'run_dom_collection.js', url]
    try:
        subprocess.run(cmd, timeout=60) #1min
    except subprocess.TimeoutExpired:
        print('run_dom_collection.js process ran too long')
        return [],None
#     os.system("node run_dom_collection.js " + url)
    
    #wait a little bit
    time.sleep(1)
    
    #close chrome
    p.terminate()
    
    print("Processing dom file")
    #process dom
    files = os.listdir("/home/gioacchino/Desktop/iogames/static_analysis/PATH_DOM_GAMEPLATFORMS/")
    if files!=[]:
        dom_file = "/home/gioacchino/Desktop/iogames/static_analysis/PATH_DOM_GAMEPLATFORMS/" + files[0]
    else:
        print("no DOM")
        return
    print(dom_file)
    
    game_info = None
    try:
        game_info = extract_poki_game_info(dom_file, url)
    except:
        print("cannot annotate game info", url)
    
    return extract_games_from_dom(dom_file), game_info



#crawling logic here -- "breadth search first" / random

#init
url = "poki.com" #default
pokigames_urls = set()

visited_urls = set()
#load alrady visited (from previous crawls)
for line in open("/home/gioacchino/Desktop/iogames/static_analysis/poki_game_info2.txt"):
    visited_urls.add(line.split("'url': '")[1].split("'")[0])
if visited_urls!=set():
    url = random.sample(visited_urls, 1)[0]
    
#start searching
while True:
    print("\n\nscraping", url)
    
    crawled_games, info = run_crawler(url)
    #annotate info on visited game
    if info!=None:
        with open(INFO,'a') as fout:
            fout.write(str(info)+'\n')

    if crawled_games!=[]:
        #add games to set
        pokigames_urls.update([g["game_url"] for g in crawled_games])
        #log to file
        with open(LOG,'a') as fout:
            for x in crawled_games:
                fout.write(x['game'] + '\t' + x['game_url'] + '\n')
        #add game to visited
        visited_urls.add(url)
    else:
        print("No result from", url)
    
    #pick random url and restart
    if pokigames_urls - visited_urls != set():
        url = random.sample(pokigames_urls - visited_urls, 1)[0]
    else:
        #saturated
        print("crawler is saturated")
        #pick a random visited one and restart
        url = random.sample(visited_urls, 1)[0]
#         break



scraping poki.com/en/g/pork-chopper
Processing website:  poki.com/en/g/pork-chopper
Processing dom file
/home/gioacchino/Desktop/iogames/static_analysis/PATH_DOM_GAMEPLATFORMS/poki.com__en__g__pork-chopper-1595470202118.flatteneddom


scraping poki.com/en/g/solitaire-golf
Processing website:  poki.com/en/g/solitaire-golf
Processing dom file
/home/gioacchino/Desktop/iogames/static_analysis/PATH_DOM_GAMEPLATFORMS/poki.com__en__g__solitaire-golf-1595470223330.flatteneddom


scraping poki.com/en/g/hex-frvr
Processing website:  poki.com/en/g/hex-frvr
Processing dom file
/home/gioacchino/Desktop/iogames/static_analysis/PATH_DOM_GAMEPLATFORMS/poki.com__en__g__hex-frvr-1595470244554.flatteneddom


scraping poki.com/en/g/mystical-birdlink
Processing website:  poki.com/en/g/mystical-birdlink
Processing dom file
/home/gioacchino/Desktop/iogames/static_analysis/PATH_DOM_GAMEPLATFORMS/poki.com__en__g__mystical-birdlink-1595470265777.flatteneddom
cannot annotate game info poki.com/en/g/mystical-bi

In [140]:
(pokigames_urls - visited_urls) == set()

True

In [133]:
pokigames_urls - visited_urls

{'poki.com/en/g/basket-ball',
 'poki.com/en/g/electric-highway',
 'poki.com/en/g/la-rex',
 'poki.com/en/g/lumber-jack-story',
 'poki.com/en/g/warscrap-io',
 'poki.comhttps:/poki.com/en/g/doctor-hero',
 'poki.comhttps:/poki.com/en/g/screw-factory',
 'poki.com“https:/poki.com/en/g/12-minibattles”',
 'poki.com“https:/poki.com/en/g/minimissions”'}

In [105]:
run_crawler("poki.com/en/g/basket-ball")

Processing website:  poki.com/en/g/basket-ball
Processing dom file
no DOM


In [110]:
import subprocess
cmd = ['echo', 'gio']
try:
    subprocess.run(cmd, timeout=5)
except subprocess.TimeoutExpired:
    print('process rexan too long')
    
print("pizza")

pizza


### Library

In [127]:
import json

def nodes_from_tree(tree, parent, r=[]):
    to_include = {x: parent[x] for x in parent if x!="children"}
    r = [to_include]
    if "children" in parent:
        for child in parent['children']:
            r.extend(nodes_from_tree(tree, child, r))
    return r

#try to extract game name, dev, likes, dislikes
def extract_poki_game_info(devtools_dom, url):
    with open(devtools_dom) as fin:
        jsondata = json.load(fin)
    res = []  
    flattendom = nodes_from_tree(jsondata, jsondata['root'])
    
    for node in flattendom:
        if "nodeType" not in node or "nodeName" not in node or "localName" not in node:
            continue
        if node["nodeType"]==3 and node["nodeName"]=="#text" and node["localName"]=="":
            res.append(node["nodeValue"])
    
#     for x in res:
#         print(x)
    #use anchor text "by "  
    anchor = res.index("by ")
    author = res[anchor+1]
    likes = res[anchor+2]
    dislikes = res[anchor+3]
    #use anchor text "games" 
    anchor = res.index("games")
    category = res[anchor+1]
    name = res[anchor+2]
    stars = res[anchor+4]
    votes = res[anchor+5]
    description = res[anchor+7]
    
    google_play = "not_found"
    apple_store = "not_found"
    
    #get google-play, apple link (if any)
    for node in flattendom:
        if "attributes" in node and "href" in node["attributes"]:
            for att in node["attributes"]:
                if "play.google" in att:
                    google_play = att
                if "apps.apple.com" in att:
                    apple_store = att
                    
    
    return {'game': name, 'url':url, 'author':author, 'category':category, 
            'votes':votes, 'likes':likes, 'dislikes':dislikes, 
            'stars':stars, 'description':description, 
            'google_play':google_play, 'apple_store':apple_store}
    
    
def extract_games_from_dom(devtools_dom):
    with open(devtools_dom) as fin:
        jsondata = json.load(fin)

    flattendom = nodes_from_tree(jsondata, jsondata['root'])
    games = []

    for node in flattendom:
        game = False
        if "attributes" in node:
            for x in node['attributes']:
                if "/en/g/" in x:
                    game = True
                    break
        if game == True:
            #collect game url
            if "http://" in x or "https://" in x:
                game_url = x.split("://")[1]
            else:
                game_url = "poki.com" + x
            #collect game name:
            game_name = game_url.split("/")[-1].replace("-", " ")
            games.append({"game":game_name, "game_url":game_url})
    return games

# extract_games_from_dom("/home/gioacchino/Desktop/iogames/static_analysis/PATH_DOM_GAMEPLATFORMS/poki.com-1594858908882.flatteneddom")


### Check how many games we have

In [4]:
import ast

raw_data = []
for line in open("./poki_game_info2.txt"):
    raw_data.append(ast.literal_eval(line))
pd.DataFrame(raw_data).drop_duplicates()

,game,url,author,category,votes,likes,dislikes,stars,description,google_play,apple_store
0,Stickman Hook,poki.com/en/g/stickman-hook,Madbox,Skill Games,"1,262,280",1.1M,169.2K,4.5,Stickman Hook is a skill game where you play a...,https://play.google.com/store/apps/details?id=...,not_found
1,ZOOM-BE 3,poki.com/en/g/zoom-be-3,Flazm,2 Player Games,"29,691",26.5K,3.2K,4.6,ZOOM-BE 3 is the latest sequel to the cooperat...,not_found,not_found
2,ZOOM-BE,poki.com/en/g/zoom-be,Flazm,2 Player Games,"145,010",126.3K,18.7K,4.5,ZOOM-BE is a cooperative game where you help t...,not_found,not_found
3,Stickman Fighter: Epic Battle,poki.com/en/g/stickman-fighter-epic-battle,Playtouch,Stickman Games,"292,223",248.3K,44.0K,4.4,Stickman Fighter: Epic Battle is an epic stick...,not_found,not_found
4,Stickman Fighter: Epic Battle 2,poki.com/en/g/stickman-fighter-epic-battle-2,Playtouch,Stickman Games,"100,307",88.4K,11.9K,4.5,Stickman Fighter: Epic Battle 2 is a stickman ...,not_found,not_found
...,...,...,...,...,...,...,...,...,...,...,...
1557,Mini Royale 2,poki.com/en/g/miniroyale-2-io,Cem Demir,.io Games,"126,278",111.7K,14.6K,4.5,"Mini Royale 2 is a battle-royale game, created...",not_found,not_found
1558,Masked Forces,poki.com/en/g/masked-forces,FreezeNova,Action Games,"37,549",32.3K,5.3K,4.4,Sneak behind enemy lines and try to survive! M...,not_found,not_found
1559,Basketball FRVR,poki.com/en/g/basketball-frvr,FRVR Games,Sports Games,"38,253",32.2K,6.0K,4.4,Basketball FRVR is a basketball game created b...,https://play.google.com/store/apps/details?id=...,not_found
1560,Basket & Ball,poki.com/en/g/basket-ball,Sun Temple,Sports Games,"18,056",14.9K,3.2K,4.3,Basket & Ball is a basketball game like no oth...,https://play.google.com/store/apps/details?id=...,not_found


### Old / Junk

In [128]:
results = extract_poki_game_info("/home/gioacchino/Desktop/iogames/static_analysis/PATH_DOM_GAMEPLATFORMS/poki.com__en__g__screw-factory-1595230788878.flatteneddom", "gg")


In [129]:
results

{'apple_store': 'https://apps.apple.com/us/app/screw-factory/id1488302065',
 'author': 'Fomo Games',
 'category': 'Simulation Games',
 'description': 'Screw Factory is a simulation game where you unleash your handyman or a handywoman. Use one of the tools inside your toolkit to assemble various objects and machines. There are loads of objects to choose from, such as toys, furniture, cars, electronics and many more! As you progress through the game, you will unlock upgrades, new tools and new objects. Be careful not to tighten the screws too hard, otherwise you will lose earnings. Can you assemble every object this addictive game has to offer?',
 'dislikes': '2.7K',
 'game': 'Screw Factory',
 'google_play': 'https://play.google.com/store/apps/details?id=jp.bap.game.screwfactory&hl=en_US',
 'likes': '17.8K',
 'stars': '4.5',
 'url': 'gg',
 'votes': '20,527'}

In [78]:
[x for x in results if x=="by "]

['by ']

In [79]:
results.index("by ")

10

In [81]:
results[11]

'Destruction Crew'

In [91]:
results[12]

'285.8K'

In [92]:
results[13]

'35.9K'

In [82]:
results.index("games")

24

In [83]:
results[25]

'Car Games'

In [84]:
results[26]

'Rocket Soccer Derby'

In [85]:
results[27]

'Destruction Crew'

In [86]:
results[28]

'4.5'

In [87]:
results[29]

'321,636'

In [88]:
results[30]

'votes'

In [89]:
results[31]

"Rocket Soccer Derby is a frantic mixture of a soccer game, a car game and a simulation game in all-in-one. You have to drive around a football pitch and try to score goals against your opponent. You can choose to play a quick game or you can choose to enter the Rocket League. By entering the Rocket League you are joing the competition and you can earn in-game money and cogs to upgrade your car! You start in the Beginners League, but once you've got the hang of it you can quickly promote to the Intermediate League and maybe you can even reach the Expert League. Give it all and win the league!"

In [90]:
results[32]

'How to play:'

In [4]:
import ast

raw_data = []
for line in open("./poki_game_info2.csv"):
    raw_data.append(ast.literal_eval(line))
pd.DataFrame(raw_data).drop_duplicates()

,game,url,author,category,votes,likes,dislikes,stars,description,google_play,apple_store
0,Stickman Hook,poki.com/en/g/stickman-hook,Madbox,Skill Games,"1,262,280",1.1M,169.2K,4.5,Stickman Hook is a skill game where you play a...,https://play.google.com/store/apps/details?id=...,not_found
1,ZOOM-BE 3,poki.com/en/g/zoom-be-3,Flazm,2 Player Games,"29,691",26.5K,3.2K,4.6,ZOOM-BE 3 is the latest sequel to the cooperat...,not_found,not_found
2,ZOOM-BE,poki.com/en/g/zoom-be,Flazm,2 Player Games,"145,010",126.3K,18.7K,4.5,ZOOM-BE is a cooperative game where you help t...,not_found,not_found
3,Stickman Fighter: Epic Battle,poki.com/en/g/stickman-fighter-epic-battle,Playtouch,Stickman Games,"292,223",248.3K,44.0K,4.4,Stickman Fighter: Epic Battle is an epic stick...,not_found,not_found
4,Stickman Fighter: Epic Battle 2,poki.com/en/g/stickman-fighter-epic-battle-2,Playtouch,Stickman Games,"100,307",88.4K,11.9K,4.5,Stickman Fighter: Epic Battle 2 is a stickman ...,not_found,not_found
...,...,...,...,...,...,...,...,...,...,...,...
1557,Mini Royale 2,poki.com/en/g/miniroyale-2-io,Cem Demir,.io Games,"126,278",111.7K,14.6K,4.5,"Mini Royale 2 is a battle-royale game, created...",not_found,not_found
1558,Masked Forces,poki.com/en/g/masked-forces,FreezeNova,Action Games,"37,549",32.3K,5.3K,4.4,Sneak behind enemy lines and try to survive! M...,not_found,not_found
1559,Basketball FRVR,poki.com/en/g/basketball-frvr,FRVR Games,Sports Games,"38,253",32.2K,6.0K,4.4,Basketball FRVR is a basketball game created b...,https://play.google.com/store/apps/details?id=...,not_found
1560,Basket & Ball,poki.com/en/g/basket-ball,Sun Temple,Sports Games,"18,056",14.9K,3.2K,4.3,Basket & Ball is a basketball game like no oth...,https://play.google.com/store/apps/details?id=...,not_found


In [5]:
df = pd.read_csv("./coolmathgames_game_info.csv", sep='\t')

,Unnamed: 0,game,url,categories,tech
0,0,1 Push,/0-1-push,"['1Push', 'HTML5', 'Logic']",HTML5
1,1,2048,/0-2048,"['2048', 'Number', 'Logic', 'Strategy', 'Home'...",HTML5
2,2,3 Pandas,/0-3-pandas,"['3Pandas', 'Logic']",Flash
3,3,3 Pandas in Fantasy,/0-3-pandas-in-fantasy,"['3PandasinFantasy', 'Logic']",Flash
4,4,31,/0-31,"['31', 'Strategy', 'HTML5', 'Number', 'Classic']",HTML5
...,...,...,...,...,...
1323,1323,Xlox,/0-xlox,"['Xlox', 'Logic', 'HTML5']",HTML5
1324,1324,Yellow,/0-yellow,"['Yellow', 'Logic']",Flash
1325,1325,Zilch,/0-zilch,"['Zilch', 'Strategy', 'Classic']",Flash
1326,1326,Zombie Launcher,/0-zombie-launcher,"['ZombieLauncher', 'HTML5']",HTML5
